# Cleaning

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
data = pd.read_csv('/kaggle/input/toxic-message-classifier-dataset/train_cleaned.csv')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_words
0,0000997932d777bf,explanation\n edits made username hardcore ...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '..."
1,000103f0d9cfb60f,daww matches background colour im seemingly ...,0,0,0,0,0,0,"['daww', 'matches', 'background', 'colour', 'i..."
2,000113f07ec002fd,hey man im really trying edit war guy c...,0,0,0,0,0,0,"['hey', 'man', 'im', 'really', 'trying', 'edit..."
3,0001b41b1c6bb37e,cant make real suggestions improvement won...,0,0,0,0,0,0,"['cant', 'make', 'real', 'suggestions', 'impro..."
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page', ..."


# Tokenizing Comments using TextVectorization

In [3]:
# finding the maximum number of words present in any given comment
maxlen = 0
longest_comment = ""
for comment in data['comment_text']:
    length = len(comment)
    if (length > maxlen):
        longest_comment = comment
    maxlen = max(maxlen, length)

print("Number of characters in the longest comment is", maxlen)
print("Number of words in the longest comment is",
      len(longest_comment.split(" ")) + 1)

Number of characters in the longest comment is 5000
Number of words in the longest comment is 456


In [4]:
import tensorflow as tf

vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(

    max_tokens=None,

    # this is greater than the max words any comment has (774)
    # the remaning spots in the output would be padded by 0s
    output_sequence_length=800,

    # converets to lowercase and skips all the punctuation
    standardize="lower_and_strip_punctuation",

    # the tokens will be split at whitespaces
    split="whitespace",

    # each of the tokens is represented as an integer
    output_mode="int",
)

In [5]:
numpyArray = data[data.columns[1]].to_numpy()
vectorize_layer.adapt(numpyArray)

In [6]:
# testing
vectorize_layer("hello, world!")

<tf.Tensor: shape=(800,), dtype=int64, numpy=
array([185, 161,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,  

# Creating a Tokenizer

### Gets all the words into one string

In [9]:
# Creates the tokenizer class
tokenizer = keras.preprocessing.text.Tokenizer()

# Combines all the words into one singular string
allWordsString = " ".join(data.head(50)["comment_text"].tolist())
allWordsList = allWordsString.split(r"\\s+")

# Updates the tokenizer with the string of all words
tokenizer.fit_on_texts(allWordsList)

# Prints word dictionary
# print(tokenizer.word_index)

# Prints length of word dictionary
print(len(tokenizer.word_index))

# Converts text to numbers
print(tokenizer.texts_to_sequences(["page", "im", "use", "mussolini"]))

1077
[[1], [2], [3], [1077]]


# Text Vectorization Layer
### Following [https://www.tensorflow.org/text/tutorials/text_classification_rnn](http://)

In [12]:
NUM_ROWS = 10000

MAX_LENGTH = None
encoder = tf.keras.layers.TextVectorization(output_sequence_length=MAX_LENGTH)
encoder.adapt(data.head(NUM_ROWS)["comment_text"].tolist())

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'article', 'page', 'wikipedia', 'would', 'talk',
       'like', 'one', 'please', 'dont', 'see', 'also', 'im', 'know',
       'think', 'edit', 'people', 'use', 'articles'], dtype='<U322')

### Testing the encoder

Encoder removes punctuation and whitespace and forces lowercase so half the cleaning we did was useless

In [13]:
commentsToEncode = data.head(3)["comment_text"]
print(commentsToEncode)

encodedComments = encoder(commentsToEncode).numpy()
print(encodedComments)

for comment in encodedComments:
    print(" ".join(vocab[comment]))

0    explanation\n  edits made   username hardcore ...
1    daww  matches  background colour im seemingly ...
2    hey man im really  trying  edit war     guy  c...
Name: comment_text, dtype: object
[[  530    53    41   437  4205 14587   257   208  2022  9650  6733  4903
   2845    44  1016 11400  2690     9    10   157   357     6     3    56
     13  3853]
 [35388  2050  1647  6725    13  3256  2365    26     6   977    95     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0]
 [  387   349    13    49   169    16   218   524  1606   406   367    30
    399    53   270     6     3   126   341  2062   506   391     0     0
      0     0]]
explanation edits made username hardcore metallica fan reverted werent vandalisms closure gas voted new york dolls fac please dont remove template talk page since im retired
daww matches background colour im seemingly stuck thanks talk january utc               
hey man im really trying edit war guy constantly

### Building the model

In [14]:
# Sets random seed so results are identical every time
SEED = 1
tf.random.set_seed(SEED)
np.random.seed(SEED)

from tensorflow.keras.callbacks import EarlyStopping

epochs = 5
learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
embedding_size = 512
lstm_size_1 = 128
conv_size_2 = 64

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=embedding_size,
        mask_zero=True,
        embeddings_regularizer=tf.keras.regularizers.L2(0.001)
    ),
    tf.keras.layers.Conv1D(128, 5, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(conv_size_2, 5, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_size_1, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# Add early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])


### Training the model

In [15]:
binaryDf = data.head(NUM_ROWS)[["comment_text", "toxic"]]

split_cutoff = int(0.8 * NUM_ROWS)
training_data = binaryDf.iloc[:split_cutoff]
validation_data = binaryDf.iloc[split_cutoff:]

training_target = training_data.pop("toxic")
validation_target = validation_data.pop("toxic")

# Test Sample Weight

In [29]:
def accForSampleWeight(weight):
    sample_weight = np.ones(shape=(len(training_target),))
    sample_weight[training_target == 1] = float(weight)

    callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=3)
    history = model.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback], sample_weight=sample_weight)
    
    test_data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test.csv")
    test_labels = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test_labels.csv")

    test_labels = test_labels.loc[test_labels["toxic"] >= 0]
    merged_df = test_labels.merge(test_data, left_on="id", right_on="id")
    
    # Tests all rows with a value of 0 or 1

    test_df = merged_df[["comment_text", "toxic"]]
    testTarget = test_df.pop("toxic")
    results = model.evaluate(test_df, testTarget)
    print(results)

## Sample Weight = 1

In [30]:
accForSampleWeight(1)

Epoch 1/10
250/250 [==============================] - 10s 38ms/step - loss: 0.0307 - accuracy: 0.9995 - val_loss: 0.4359 - val_accuracy: 0.9305
Epoch 2/10
250/250 [==============================] - 9s 36ms/step - loss: 0.0258 - accuracy: 0.9998 - val_loss: 0.4353 - val_accuracy: 0.9350
Epoch 3/10
250/250 [==============================] - 9s 36ms/step - loss: 0.0243 - accuracy: 0.9998 - val_loss: 0.4710 - val_accuracy: 0.9315
Epoch 4/10
250/250 [==============================] - 9s 38ms/step - loss: 0.0213 - accuracy: 1.0000 - val_loss: 0.4741 - val_accuracy: 0.9350
Epoch 5/10
250/250 [==============================] - 9s 37ms/step - loss: 0.0326 - accuracy: 0.9966 - val_loss: 0.3698 - val_accuracy: 0.9425
Epoch 6/10
250/250 [==============================] - 10s 38ms/step - loss: 0.0373 - accuracy: 0.9956 - val_loss: 0.3646 - val_accuracy: 0.9410
Epoch 7/10
2000/2000 [==============================] - 37s 18ms/step - loss: 0.5256 - accuracy: 0.8950
[0.5255556702613831, 0.8949951529502

## Sample Weight 5

In [31]:
accForSampleWeight(5)

Epoch 1/10
250/250 [==============================] - 10s 39ms/step - loss: 0.0422 - accuracy: 0.9964 - val_loss: 0.3595 - val_accuracy: 0.9445
Epoch 2/10
250/250 [==============================] - 10s 39ms/step - loss: 0.0470 - accuracy: 0.9965 - val_loss: 0.4046 - val_accuracy: 0.9285
Epoch 3/10
250/250 [==============================] - 9s 38ms/step - loss: 0.0390 - accuracy: 0.9979 - val_loss: 0.4231 - val_accuracy: 0.9270
Epoch 4/10
250/250 [==============================] - 10s 38ms/step - loss: 0.0309 - accuracy: 0.9990 - val_loss: 0.4785 - val_accuracy: 0.9330
Epoch 5/10
250/250 [==============================] - 9s 38ms/step - loss: 0.0257 - accuracy: 1.0000 - val_loss: 0.5324 - val_accuracy: 0.9245
Epoch 6/10
250/250 [==============================] - 9s 36ms/step - loss: 0.0236 - accuracy: 1.0000 - val_loss: 0.5117 - val_accuracy: 0.9345
Epoch 7/10
250/250 [==============================] - 9s 38ms/step - loss: 0.0220 - accuracy: 0.9999 - val_loss: 0.5041 - val_accuracy: 0.9

# Sample Weight 10

In [37]:
accForSampleWeight(10)

Epoch 1/10
250/250 [==============================] - 9s 38ms/step - loss: 0.0346 - accuracy: 0.9998 - val_loss: 0.6861 - val_accuracy: 0.8910
Epoch 2/10
250/250 [==============================] - 10s 39ms/step - loss: 0.0319 - accuracy: 0.9998 - val_loss: 0.6680 - val_accuracy: 0.8960
Epoch 3/10
250/250 [==============================] - 9s 38ms/step - loss: 0.0322 - accuracy: 0.9995 - val_loss: 0.6036 - val_accuracy: 0.9065
Epoch 4/10
2000/2000 [==============================] - 36s 18ms/step - loss: 3.9227 - accuracy: 0.3898
[3.922663450241089, 0.3897746205329895]


## Sample Weight 2

In [39]:
accForSampleWeight(2)

Epoch 1/10
250/250 [==============================] - 10s 39ms/step - loss: 0.0265 - accuracy: 0.9999 - val_loss: 0.6260 - val_accuracy: 0.9065
Epoch 2/10
250/250 [==============================] - 9s 36ms/step - loss: 0.0244 - accuracy: 0.9999 - val_loss: 0.6348 - val_accuracy: 0.9040
Epoch 3/10
250/250 [==============================] - 9s 36ms/step - loss: 0.0236 - accuracy: 0.9999 - val_loss: 0.6337 - val_accuracy: 0.8970
Epoch 4/10
250/250 [==============================] - 10s 40ms/step - loss: 0.0214 - accuracy: 1.0000 - val_loss: 0.6404 - val_accuracy: 0.9075
Epoch 5/10
250/250 [==============================] - 9s 36ms/step - loss: 0.0227 - accuracy: 0.9995 - val_loss: 0.6011 - val_accuracy: 0.9095
Epoch 6/10
250/250 [==============================] - 10s 39ms/step - loss: 0.0281 - accuracy: 0.9981 - val_loss: 0.5464 - val_accuracy: 0.9075
Epoch 7/10
2000/2000 [==============================] - 37s 19ms/step - loss: 1.9255 - accuracy: 0.5467
[1.9255070686340332, 0.546672284603

# Sample Weight 1.5

In [40]:
accForSampleWeight(1.5)

Epoch 1/10
250/250 [==============================] - 10s 39ms/step - loss: 0.0362 - accuracy: 0.9976 - val_loss: 0.4598 - val_accuracy: 0.9140
Epoch 2/10
250/250 [==============================] - 9s 37ms/step - loss: 0.0288 - accuracy: 0.9994 - val_loss: 0.4628 - val_accuracy: 0.9190
Epoch 3/10
250/250 [==============================] - 9s 38ms/step - loss: 0.0237 - accuracy: 0.9996 - val_loss: 0.4655 - val_accuracy: 0.9250
Epoch 4/10
250/250 [==============================] - 9s 35ms/step - loss: 0.0200 - accuracy: 1.0000 - val_loss: 0.5024 - val_accuracy: 0.9225
Epoch 5/10
250/250 [==============================] - 9s 36ms/step - loss: 0.0187 - accuracy: 0.9999 - val_loss: 0.5028 - val_accuracy: 0.9285
Epoch 6/10
250/250 [==============================] - 9s 37ms/step - loss: 0.0181 - accuracy: 0.9999 - val_loss: 0.5776 - val_accuracy: 0.9165
Epoch 7/10
2000/2000 [==============================] - 37s 18ms/step - loss: 3.2875 - accuracy: 0.4104
[3.287536144256592, 0.410406708717346

# Sample Weight 100

In [36]:
accForSampleWeight(100)

Epoch 1/10
250/250 [==============================] - 10s 38ms/step - loss: 0.1055 - accuracy: 0.9966 - val_loss: 0.7040 - val_accuracy: 0.8745
Epoch 2/10
250/250 [==============================] - 9s 35ms/step - loss: 0.0791 - accuracy: 0.9981 - val_loss: 0.6458 - val_accuracy: 0.8860
Epoch 3/10
250/250 [==============================] - 9s 38ms/step - loss: 0.0639 - accuracy: 0.9991 - val_loss: 0.6814 - val_accuracy: 0.8790
Epoch 4/10
250/250 [==============================] - 9s 36ms/step - loss: 0.0574 - accuracy: 0.9994 - val_loss: 0.7209 - val_accuracy: 0.8790
Epoch 5/10
250/250 [==============================] - 10s 39ms/step - loss: 0.0527 - accuracy: 0.9994 - val_loss: 0.6854 - val_accuracy: 0.8840
Epoch 6/10
250/250 [==============================] - 9s 37ms/step - loss: 0.0485 - accuracy: 0.9995 - val_loss: 0.7315 - val_accuracy: 0.8840
Epoch 7/10
250/250 [==============================] - 9s 38ms/step - loss: 0.0454 - accuracy: 0.9995 - val_loss: 0.7622 - val_accuracy: 0.88